# Notebook 02: EDA and Normalisation

This notebook performs exploratory data analysis (EDA) and light normalisation on the document level dataset produced in Notebook 01.

## Inputs

This notebook expects the following processed files from Notebook 01:

- `data/processed/docs_extracted.csv`
- `data/processed/docs_extracted.json` (used for inspection if needed)

Each row in `docs_extracted.csv` represents a single synthetic insurance related document with fields such as:

- `entity_name`
- `region`
- `sector`
- `risk_type`
- `time_horizon`
- `key_risk_factors`
- `risk_summary`
- document level metadata (for example filename and category)

## Goals of this notebook

The main goals are to:

1. Load the extracted document level dataset into pandas.
2. Perform high level EDA on the structured fields:
   - basic distributions for `region`, `sector`, `risk_type`, `time_horizon`
   - counts and patterns by document category (policies, esg, incidents)
3. Inspect missingness and simple data quality issues.
4. Analyse text lengths:
   - length of `risk_summary` in characters and words
   - simple checks on the number of `key_risk_factors` per document
5. Create a small set of derived numeric features:
   - `num_risk_factors`
   - `summary_length_words`
   - `summary_length_chars`
6. Apply light normalisation to categorical fields where needed (for example consistent naming and ordering).

The focus here is understanding the dataset and preparing clean features, not heavy cleaning or modelling.

## Outputs

At the end of this notebook we will produce:

- A clean document level DataFrame ready for modelling.
- A saved processed file:

  - `data/processed/docs_clean.csv`

This will be the main input to Notebook 03, where we will build and evaluate a simple High Risk vs Low Risk classifier on top of these features.


## Step 1: Load the extracted document level dataset

In this step we load the output of Notebook 01, which is the file `docs_extracted.csv` stored in the `data/processed/` folder.

This dataset contains one row per document with fields extracted by the LLM, including:

- categorical fields such as `region`, `sector`, `risk_type`, `time_horizon`
- text fields such as `risk_summary`
- list based fields such as `key_risk_factors`

Once loaded, we will inspect the basic structure of the DataFrame to confirm:

- the number of documents
- column names
- presence of any missing values
- a quick preview of the first few rows

This gives us a consistent starting point for further EDA in the next steps.


In [1]:
# Step 1: Load the extracted document level dataset

from pathlib import Path
import pandas as pd

# Define project root and processed data directory
PROJECT_ROOT = Path("..").resolve()
DATA_PROCESSED_DIR = PROJECT_ROOT / "data" / "processed"

# Path to the extracted document level CSV from Notebook 01
DOCS_EXTRACTED_PATH = DATA_PROCESSED_DIR / "docs_extracted.csv"

# Load the dataset
docs_df = pd.read_csv(DOCS_EXTRACTED_PATH)

# Basic checks
print("Shape of docs_df:", docs_df.shape)
print("\nColumn names:")
print(docs_df.columns.tolist())

print("\nData types and non null counts:")
print(docs_df.info())

print("\nPreview of the first few rows:")
docs_df.head()


Shape of docs_df: (14, 12)

Column names:
['entity_name', 'region', 'sector', 'risk_type', 'time_horizon', 'key_risk_factors', 'risk_summary', 'doc_index', 'category', 'filename', 'n_chunks', 'n_chunks_with_data']

Data types and non null counts:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   entity_name         10 non-null     object
 1   region              14 non-null     object
 2   sector              14 non-null     object
 3   risk_type           14 non-null     object
 4   time_horizon        14 non-null     object
 5   key_risk_factors    14 non-null     object
 6   risk_summary        14 non-null     object
 7   doc_index           14 non-null     int64 
 8   category            14 non-null     object
 9   filename            14 non-null     object
 10  n_chunks            14 non-null     int64 
 11  n_chunks_with_data  1

,entity_name,region,sector,risk_type,time_horizon,key_risk_factors,risk_summary,doc_index,category,filename,n_chunks,n_chunks_with_data
0,"global manufacturing, logistics, and energy co...",global,industrial,esg,not_specified,"['inconsistent ESG strategy', 'different inter...",The company's ESG strategy is still developing...,0,esg,esg_corporate_sustainability.txt,4,4
1,The organisation,global,"energy, logistics, heavy industry, transport",esg,medium_term,['inconsistency in language used in internal d...,The organisation's energy transition strategy ...,1,esg,esg_energy_transition.txt,5,5
2,Synthetic ESG Report – Supply Chain and Climat...,global,energy,esg,not_specified,['inconsistent expectations in environmental c...,The company's supply chain faces risks related...,2,esg,esg_supply_chain_governance.txt,5,5
3,NaN,global,marine,property,not_specified,"['engine failure', 'grounding', 'irregular vib...",The incident involved a synthetic marine vesse...,3,incident,incident_marine_grounding.txt,4,4
4,Motor Fleet Collision Event,global,transportation,motor,not_specified,"['driver error', 'vehicle maintenance', 'senso...",A motor fleet collision event occurred on a du...,4,incident,incident_motor_fleet_collision.txt,4,4


## Step 2: High level overview and basic distributions

In this step we take a first look at the structure of the dataset to understand how information is spread across the 14 documents. This involves simple exploratory checks such as:

- counts of documents per category (policy, esg, incident)
- frequency of values in key extracted fields:
  - `region`
  - `sector`
  - `risk_type`
  - `time_horizon`

These checks help verify that the extraction pipeline produced reasonable variation and that there are no unexpected values. They also give an initial sense of the dataset before we examine missingness, text lengths, and risk factor patterns in later steps.

This step focuses only on simple counts and distributions to build an intuition for the dataset.


In [2]:
# Step 2: High level overview and basic distributions

# 1. Count documents per category (policy, esg, incident)
print("Document counts by category:\n")
print(docs_df["category"].value_counts())
print("\nRelative frequencies by category:\n")
print(docs_df["category"].value_counts(normalize=True))

print("\n" + "-" * 60 + "\n")

# 2. Frequency of regions
print("Region value counts:\n")
print(docs_df["region"].value_counts())
print("\nRegion relative frequencies:\n")
print(docs_df["region"].value_counts(normalize=True))

print("\n" + "-" * 60 + "\n")

# 3. Frequency of sectors
print("Sector value counts:\n")
print(docs_df["sector"].value_counts())
print("\nSector relative frequencies:\n")
print(docs_df["sector"].value_counts(normalize=True))

print("\n" + "-" * 60 + "\n")

# 4. Frequency of risk types
print("Risk type value counts:\n")
print(docs_df["risk_type"].value_counts())
print("\nRisk type relative frequencies:\n")
print(docs_df["risk_type"].value_counts(normalize=True))

print("\n" + "-" * 60 + "\n")

# 5. Frequency of time horizons
print("Time horizon value counts:\n")
print(docs_df["time_horizon"].value_counts())
print("\nTime horizon relative frequencies:\n")
print(docs_df["time_horizon"].value_counts(normalize=True))


Document counts by category:

category
policy      8
esg         3
incident    3
Name: count, dtype: int64

Relative frequencies by category:

category
policy      0.571429
esg         0.214286
incident    0.214286
Name: proportion, dtype: float64

------------------------------------------------------------

Region value counts:

region
global           11
north_america     3
Name: count, dtype: int64

Region relative frequencies:

region
global           0.785714
north_america    0.214286
Name: proportion, dtype: float64

------------------------------------------------------------

Sector value counts:

sector
insurance                                       6
travel                                          2
industrial                                      1
energy, logistics, heavy industry, transport    1
marine                                          1
energy                                          1
manufacturing                                   1
transportation               

## Step 3: Missingness and data quality checks

In this step we examine the dataset for missing values and potential data quality issues. The goal is to understand which fields are reliable, which fields need light normalisation, and which fields should not be used directly without processing.

Key checks in this step:

- Identify columns with missing values, especially `entity_name`
- Inspect whether list based fields such as `key_risk_factors` have been stored as strings
- Examine the consistency and completeness of the `risk_summary` text
- Check for placeholder or default values such as `global` in `region` or `not_specified` in `time_horizon`
- Look for unusual or unexpected categories that may need correction

These checks help us decide how much cleaning or normalisation is needed before moving to feature engineering. The objective is not heavy cleaning, but building a clear understanding of the strengths and weaknesses of each field.


In [3]:
# Step 3: Missingness and data quality checks

# 1. Missing values per column
print("Missing values per column:\n")
missing_counts = docs_df.isna().sum()
print(missing_counts)

print("\nProportion missing per column:\n")
missing_props = docs_df.isna().mean()
print(missing_props)

print("\n" + "-" * 60 + "\n")

# 2. Inspect placeholder style values in key extracted fields
for col in ["region", "time_horizon"]:
    print(f"Value counts for {col}:\n")
    print(docs_df[col].value_counts())
    print("\nRelative frequencies:\n")
    print(docs_df[col].value_counts(normalize=True))
    print("\n" + "-" * 60 + "\n")

# 3. Inspect entity_name values, including missing ones
print("Sample entity_name values (including NaNs):\n")
print(docs_df["entity_name"].head(10))

print("\n" + "-" * 60 + "\n")

# 4. Inspect key_risk_factors format
print("Type of a key_risk_factors entry:", type(docs_df.loc[0, "key_risk_factors"]))
print("\nSample key_risk_factors values:\n")
print(docs_df["key_risk_factors"].head(5))

print("\n" + "-" * 60 + "\n")

# 5. Basic properties of risk_summary
summary_lengths = docs_df["risk_summary"].str.len()

print("Risk summary length stats (characters):\n")
print(summary_lengths.describe())

print("\nSample risk_summary values:\n")
for i, text in docs_df["risk_summary"].head(3).items():
    print(f"\nDocument {i}, length {len(text)} characters:\n")
    print(text[:200], "...")


Missing values per column:

entity_name           4
region                0
sector                0
risk_type             0
time_horizon          0
key_risk_factors      0
risk_summary          0
doc_index             0
category              0
filename              0
n_chunks              0
n_chunks_with_data    0
dtype: int64

Proportion missing per column:

entity_name           0.285714
region                0.000000
sector                0.000000
risk_type             0.000000
time_horizon          0.000000
key_risk_factors      0.000000
risk_summary          0.000000
doc_index             0.000000
category              0.000000
filename              0.000000
n_chunks              0.000000
n_chunks_with_data    0.000000
dtype: float64

------------------------------------------------------------

Value counts for region:

region
global           11
north_america     3
Name: count, dtype: int64

Relative frequencies:

region
global           0.785714
north_america    0.214286
Name: 

## Step 4: Convert fields and create early derived features

Before moving into deeper EDA, we need to convert a few columns into cleaner formats and create some simple numeric features that will support modelling later.

In this step we will:

1. Convert the `key_risk_factors` column into a real Python list.  
   The values in this column come from LLM extraction and are stored as long, irregular strings.  
   Instead of using strict parsing, we use a simple and robust regex based approach that:
   - extracts every substring inside single quotes `'...'`
   - removes any extra whitespace
   - always returns a valid Python list  
   This gives us a clean list of risk factor phrases for each document.

2. Create basic derived numeric features:
   - `num_risk_factors`: number of extracted risk factor phrases  
   - `summary_length_chars`: character length of the risk_summary text  
   - `summary_length_words`: word count of the risk_summary text

These early features capture a sense of document complexity and variation across the dataset.  
They will be useful both for EDA in this notebook and for model building in Notebook 03.


In [4]:
# Step 4: Convert fields and create early derived features

import re

# 1. Robust parser for key_risk_factors
def extract_risk_factors(value):
    """
    Convert the key_risk_factors string into a clean Python list.
    Uses regex to extract everything between single quotes.
    Always returns a Python list (possibly empty).
    """
    if isinstance(value, list):
        return value
    if pd.isna(value):
        return []
    
    text = str(value)
    
    # Extract all substrings inside single quotes '...'
    items = re.findall(r"'([^']+)'", text)
    
    # Strip extra whitespace and return
    return [item.strip() for item in items]

# Apply parser to create a clean list column
docs_df["key_risk_factors_list"] = docs_df["key_risk_factors"].apply(extract_risk_factors)

# 2. Create derived numeric features

# Number of key risk factors
docs_df["num_risk_factors"] = docs_df["key_risk_factors_list"].apply(len)

# Length of risk_summary in characters
docs_df["summary_length_chars"] = docs_df["risk_summary"].str.len()

# Length of risk_summary in words
docs_df["summary_length_words"] = docs_df["risk_summary"].str.split().str.len()

# Quick check of the new columns
print("Preview of derived features:\n")
print(
    docs_df[
        [
            "doc_index",
            "category",
            "risk_type",
            "num_risk_factors",
            "summary_length_chars",
            "summary_length_words",
        ]
    ].head()
)

docs_df["key_risk_factors_list"].head(10)


Preview of derived features:

   doc_index  category risk_type  num_risk_factors  summary_length_chars  \
0          0       esg       esg                28                   804   
1          1       esg       esg                20                   804   
2          2       esg       esg                19                   804   
3          3  incident  property                21                   804   
4          4  incident     motor                14                   804   

   summary_length_words  
0                   115  
1                   108  
2                   106  
3                   117  
4                   128  


0    [inconsistent ESG strategy, different interpre...
1    [inconsistency in language used in internal do...
2    [inconsistent expectations in environmental cr...
3    [engine failure, grounding, irregular vibratio...
4    [driver error, vehicle maintenance, sensor cal...
5    [faulty fire alarm panel, unusual smell, contr...
6    [bodily injury, property damage, liability, fa...
7    [exclusions, expected losses, expected or inte...
8    [malicious attacks, accidental system failures...
9    [discrepancies in reported earnings, late enro...
Name: key_risk_factors_list, dtype: object

## Step 5: Explore derived features and relationships

In this step we use the new derived features to deepen our understanding of the documents and how risk information is distributed.

We will focus on:

1. Distributions of the derived numeric features:
   - `num_risk_factors`
   - `summary_length_chars`
   - `summary_length_words`

2. Simple relationships between these features and key categorical fields:
   - by `category` (policy, esg, incident)
   - by `risk_type` (property, motor, cyber, travel, liability, esg)

Examples of questions we want to answer:

- Do incident or ESG style documents tend to have more key risk factors than policies?
- Are certain risk types associated with longer summaries or more risk factors?
- Is there enough variation in these features to be useful later in modelling?

We will use simple descriptive statistics and grouped summaries (for example `groupby().agg(...)`) to build intuition before moving on to normalisation and final cleaning.


In [5]:
# Step 5: Explore derived features and relationships

# 1. Distribution of numeric features
print("Summary of numeric derived features:\n")
print(
    docs_df[
        ["num_risk_factors", "summary_length_chars", "summary_length_words"]
    ].describe()
)

print("\n" + "-" * 60 + "\n")

# 2. Compare derived features by category (policy, esg, incident)
print("Derived feature means by category:\n")
print(
    docs_df.groupby("category")[
        ["num_risk_factors", "summary_length_chars", "summary_length_words"]
    ].mean()
)

print("\n" + "-" * 60 + "\n")

# 3. Compare derived features by risk_type
print("Derived feature means by risk_type:\n")
print(
    docs_df.groupby("risk_type")[
        ["num_risk_factors", "summary_length_chars", "summary_length_words"]
    ].mean()
)

print("\n" + "-" * 60 + "\n")

# 4. Optional: show variation with simple scatter style table
print("Quick comparison table of key fields:\n")
print(
    docs_df[
        ["category", "risk_type", "num_risk_factors", "summary_length_words"]
    ].sort_values("num_risk_factors", ascending=False)
)


Summary of numeric derived features:

       num_risk_factors  summary_length_chars  summary_length_words
count         14.000000             14.000000             14.000000
mean          21.000000            800.285714            112.642857
std            7.421383             13.612373              7.438022
min           11.000000            753.000000            102.000000
25%           14.250000            804.000000            108.000000
50%           20.000000            804.000000            112.500000
75%           25.000000            804.000000            116.500000
max           36.000000            804.000000            128.000000

------------------------------------------------------------

Derived feature means by category:

          num_risk_factors  summary_length_chars  summary_length_words
category                                                              
esg              22.333333                 804.0            109.666667
incident         15.333333            

## Step 6: Light normalisation and final cleaning

Before saving the cleaned dataset, we apply a small number of normalisation steps to improve consistency and prepare the data for modelling in Notebook 03.

In this step we will:

1. Standardise categorical fields where simple inconsistencies appear  
   (for example lowercasing or trimming whitespace if needed).

2. Ensure that controlled vocabulary fields such as  
   `risk_type`, `region`, and `time_horizon` follow the same spelling and format  
   used during extraction.

3. Inspect the `sector` field for minor inconsistencies and decide whether  
   to normalise or keep as-is.  
   Since sectors vary widely across documents, we will avoid heavy cleaning  
   and keep the original labels unless there are obvious duplicates.

4. Remove or reorder columns that are not useful for modelling  
   (for example internal metadata such as `filename` or `n_chunks`).

5. Prepare the final set of columns to be saved as  
   `data/processed/docs_clean.csv`, which will be the main input to  
   Notebook 03 for feature engineering and classification.

This step focuses on minimal, safe adjustments rather than aggressive cleaning,  
since the dataset is small and the extraction pipeline already enforces structure.  
The goal is simply to ensure consistency and produce a tidy DataFrame ready for modelling.


In [6]:
# Step 6: Light normalisation and final cleaning

# 1. Strip leading/trailing whitespace from all string-like columns
str_cols = docs_df.select_dtypes(include="object").columns.tolist()

# Exclude the list column, which is stored as object dtype but contains Python lists
str_cols = [col for col in str_cols if col != "key_risk_factors_list"]

for col in str_cols:
    docs_df[col] = docs_df[col].str.strip()

# 2. Enforce consistent lowercase for simple controlled vocab fields
for col in ["region", "risk_type", "time_horizon", "category"]:
    docs_df[col] = docs_df[col].str.lower()

# 3. Fix missing entity_name
docs_df["entity_name"] = docs_df["entity_name"].fillna("unspecified_entity")

# 4. Decide which columns to keep for modelling
columns_to_keep = [
    "doc_index",
    "category",
    "entity_name",
    "region",
    "sector",
    "risk_type",
    "time_horizon",
    "key_risk_factors_list",
    "num_risk_factors",
    "summary_length_chars",
    "summary_length_words",
    "risk_summary",
]

docs_clean = docs_df[columns_to_keep].copy()

# 5. Save the cleaned dataset
CLEAN_PATH = DATA_PROCESSED_DIR / "docs_clean.csv"
docs_clean.to_csv(CLEAN_PATH, index=False)

print("Saved cleaned dataset to:", CLEAN_PATH)
print("\nShape of docs_clean:", docs_clean.shape)
print("\nColumns in docs_clean:\n", docs_clean.columns.tolist())
print("\nPreview of docs_clean:\n")
docs_clean.head()


Saved cleaned dataset to: C:\Users\misha\OneDrive - University of Bristol\Job Apps\Concirrus\genai-insurance-risk-extraction\data\processed\docs_clean.csv

Shape of docs_clean: (14, 12)

Columns in docs_clean:
 ['doc_index', 'category', 'entity_name', 'region', 'sector', 'risk_type', 'time_horizon', 'key_risk_factors_list', 'num_risk_factors', 'summary_length_chars', 'summary_length_words', 'risk_summary']

Preview of docs_clean:



,doc_index,category,entity_name,region,sector,risk_type,time_horizon,key_risk_factors_list,num_risk_factors,summary_length_chars,summary_length_words,risk_summary
0,0,esg,"global manufacturing, logistics, and energy co...",global,industrial,esg,not_specified,"[inconsistent ESG strategy, different interpre...",28,804,115,The company's ESG strategy is still developing...
1,1,esg,The organisation,global,"energy, logistics, heavy industry, transport",esg,medium_term,[inconsistency in language used in internal do...,20,804,108,The organisation's energy transition strategy ...
2,2,esg,Synthetic ESG Report – Supply Chain and Climat...,global,energy,esg,not_specified,[inconsistent expectations in environmental cr...,19,804,106,The company's supply chain faces risks related...
3,3,incident,unspecified_entity,global,marine,property,not_specified,"[engine failure, grounding, irregular vibratio...",21,804,117,The incident involved a synthetic marine vesse...
4,4,incident,Motor Fleet Collision Event,global,transportation,motor,not_specified,"[driver error, vehicle maintenance, sensor cal...",14,804,128,A motor fleet collision event occurred on a du...


## Summary of Notebook 02: EDA and Normalisation

In this notebook we explored the extracted document level dataset produced in Notebook 01 and prepared it for modelling in the next stage.

Key steps completed:

1. **Loaded the extracted dataset**  
   Imported `docs_extracted.csv` and inspected overall shape, schema, and field types.

2. **Performed high level EDA**  
   - Checked distributions of categories, regions, sectors, and risk types.  
   - Investigated missing values and confirmed that only `entity_name` required imputation.  
   - Examined variation in summary lengths and risk factor counts.

3. **Converted raw fields into clean formats**  
   - Used a robust regex based method to convert `key_risk_factors` into real Python lists.  
   - Created early derived features including  
     `num_risk_factors`, `summary_length_chars`, and `summary_length_words`.

4. **Explored relationships in the derived features**  
   - Compared averages by `category` and `risk_type`.  
   - Observed clear and meaningful variation that will be useful for modelling.

5. **Light normalisation and final cleaning**  
   - Stripped whitespace from text fields.  
   - Lowercased consistent categorical fields.  
   - Filled missing `entity_name` values with a clean placeholder.  
   - Selected and saved a tidy set of modelling ready columns.

6. **Saved the cleaned dataset**  
   Exported the final dataset to  
   `data/processed/docs_clean.csv`, which will be the input to Notebook 03.

The dataset is now clean, consistent, and ready for feature engineering and model building.  
The next notebook will focus on constructing features, handling categorical variables, building baseline models, and interpreting the results in a risk scoring context.
